## Exploring variation of channel positions and mapping across runs and participants

### Set-up
Load datasets and file paths of associated runs and participants 

In [1]:
import sys
sys.path.append('..')

from dataset.dataSet import OpenFMRIDataSet
from dotenv import load_dotenv
import os

load_dotenv()
dataset_path = os.getenv("DATASET_PATH")


### init dataset this will automatically download and preprocess the dataset if run for the first time
train_dataset = OpenFMRIDataSet(mode='train',datasetPath=dataset_path)
val_dataset = OpenFMRIDataSet(mode='val',datasetPath=dataset_path,loadData=False)
test_dataset = OpenFMRIDataSet(mode='test',datasetPath=dataset_path,loadData=False) 


train_dir = train_dataset.datasetPath
val_dir = val_dataset.datasetPath
test_dir = test_dataset.datasetPath

train_subject_run_dict = train_dataset._participantRunsDict
val_subject_run_dict = val_dataset._participantRunsDict
test_subject_run_dict = test_dataset._participantRunsDict


INFO:dataset.dataSet:Skipping downloading and preparing... (Found .downloaded, .unzipped, .arranged, .randomized files)
INFO:dataset.dataSet:

Loading dataset in train mode...
INFO:dataset.dataSet:Window length (No. frames): 275
INFO:dataset.dataSet:Cache files found. 183720 loaded from cache.
INFO:dataset.dataSet:

Loading dataset in val mode...
INFO:dataset.dataSet:Window length (No. frames): 275
INFO:dataset.dataSet:Cache files found. 48026 loaded from cache.
INFO:dataset.dataSet:

Loading dataset in test mode...
INFO:dataset.dataSet:Window length (No. frames): 275
INFO:dataset.dataSet:Cache files found. 33983 loaded from cache.


In [4]:
train_dataset[0][1].shape

(102, 275)

In [54]:
mode_dicts = [val_subject_run_dict]#,train_subject_run_dict,test_subject_run_dict]
mode_dirs = [val_dir]#,train_dir,test_dir]

fif_files = []

### iterate through and collect fif files of every run of every participant of every folder (train, test, val)
for mode_dict, mode_dir in zip(mode_dicts,mode_dirs):
    for subject in mode_dict:
        for run_file in mode_dict[subject]:
            fif_file = os.path.join(mode_dir,subject,run_file)
            fif_files.append(fif_file)
fif_files

['/srv/synaptech_openfmri/val/sub-03/run_06.fif',
 '/srv/synaptech_openfmri/val/sub-03/run_05.fif',
 '/srv/synaptech_openfmri/val/sub-03/run_01.fif',
 '/srv/synaptech_openfmri/val/sub-03/run_04.fif',
 '/srv/synaptech_openfmri/val/sub-03/run_03.fif',
 '/srv/synaptech_openfmri/val/sub-11/run_06.fif',
 '/srv/synaptech_openfmri/val/sub-11/run_05.fif',
 '/srv/synaptech_openfmri/val/sub-11/run_01.fif',
 '/srv/synaptech_openfmri/val/sub-11/run_04.fif',
 '/srv/synaptech_openfmri/val/sub-11/run_02.fif',
 '/srv/synaptech_openfmri/val/sub-11/run_03.fif',
 '/srv/synaptech_openfmri/val/sub-09/run_06.fif',
 '/srv/synaptech_openfmri/val/sub-09/run_05.fif',
 '/srv/synaptech_openfmri/val/sub-09/run_01.fif',
 '/srv/synaptech_openfmri/val/sub-09/run_04.fif',
 '/srv/synaptech_openfmri/val/sub-09/run_02.fif',
 '/srv/synaptech_openfmri/val/sub-09/run_03.fif']

### Plotting channel locations
First, get channel coordinates and channel names

In [55]:
import mne
import warnings
import contextlib
import numpy as np


eeg_indices = train_dataset.eeg_indices
meg_indices = train_dataset.meg_indices
assert type(eeg_indices[0]) == int, "eeg index not an int"
assert np.array_equal(eeg_indices, test_dataset.eeg_indices) and np.array_equal(eeg_indices, val_dataset.eeg_indices), "EEG indices are not equal"
assert np.array_equal(meg_indices, test_dataset.meg_indices) and np.array_equal(meg_indices, val_dataset.meg_indices), "MEG indices are not equal"

run_names = []

all_channel_names = []
all_eeg_channel_names = []
all_meg_channel_names = []

all_channel_locs = []
all_eeg_channel_locs = []
all_meg_channel_locs = []

all_meg_transforms = []

for run in fif_files:
    run_name = run ## TODO clean name
    run_names.append(run_name)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with open(os.devnull, 'w') as f, contextlib.redirect_stdout(f):    
            raw = mne.io.read_raw_fif(run, preload=False)
    channels = raw.info['chs']

    all_meg_transforms.append(raw.info['dev_head_t']) ## get transformation matrix device->head

    channel_locs = []
    channel_names = []
    eeg_channel_names = []
    meg_channel_names = []
    
    for channel in channels:
        channel_names.append(channel['ch_name'])
        channel_locs.append([channel['loc'][0],channel['loc'][1],channel['loc'][2]])
     
    meg_channel_locs = np.array(channel_locs)[meg_indices]
    eeg_channel_locs = np.array(channel_locs)[eeg_indices]
    
    eeg_channel_names = np.array(channel_names)[eeg_indices].tolist()
    meg_channel_names = np.array(channel_names)[meg_indices].tolist()
    
    all_eeg_channel_locs.append(eeg_channel_locs)
    all_meg_channel_locs.append(meg_channel_locs)
    all_eeg_channel_names.append(eeg_channel_names)
    all_meg_channel_names.append(meg_channel_names)
    all_channel_locs.append(channel_locs)
    all_channel_names.append(channel_names)

In [56]:
print(all_meg_channel_locs)

[array([[-1.06600001e-01,  4.63999994e-02, -6.04000017e-02],
       [-1.01999998e-01,  6.31000027e-02, -2.55999994e-02],
       [-1.08499996e-01,  3.02000009e-02, -2.65999995e-02],
       [-1.09899998e-01,  1.31000001e-02, -6.27000034e-02],
       [-1.07400000e-01,  3.29000019e-02,  8.00000038e-03],
       [-9.88999978e-02,  4.03000005e-02,  4.12999988e-02],
       [-1.01099998e-01,  4.39999998e-03,  4.08000015e-02],
       [-1.08300000e-01, -1.09999999e-03,  7.10000005e-03],
       [-8.60999972e-02,  9.88000035e-02,  8.99999961e-03],
       [-8.86999965e-02,  7.56999999e-02,  4.12000008e-02],
       [-7.02000037e-02,  7.58000016e-02,  7.06999972e-02],
       [-1.00299999e-01,  6.58999979e-02,  8.10000021e-03],
       [-8.07999969e-02,  4.12999988e-02,  7.19999969e-02],
       [-5.26000001e-02,  4.06000018e-02,  9.52000022e-02],
       [-5.37000000e-02,  5.90000022e-03,  9.69000012e-02],
       [-8.29000026e-02,  6.20000018e-03,  7.28000030e-02],
       [-6.36999980e-02,  1.25400007e-0

In [57]:
import numpy as np
import plotly.graph_objects as go
import mne

def plotSensors(all_eeg_channel_locs :list , all_meg_channel_locs:list, megToEEGtransforms = None, sphere = False, output_name= "channel_plot.html"):
    """
        Takes a list of channel_locs, each of which is a list of 3D coordinates.
    """
    # Create a sphere to simulate the head
    phi, theta = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
    x = 0.1 * np.sin(theta) * np.cos(phi)  # Radius scaled for visualization
    y = 0.1 * np.sin(theta) * np.sin(phi)
    z = 0.1 * np.cos(theta)

    # Create the plotly figure
    fig = go.Figure()
    
    if sphere: # Plot the head (sphere)
        fig.add_trace(go.Surface(x=x, y=y, z=z, colorscale='Blues', opacity=0.1, showscale=False))

    for eeg_channel_locs in all_eeg_channel_locs:
        # Plot EEG channels
        fig.add_trace(go.Scatter3d(x=eeg_channel_locs[:, 0], y=eeg_channel_locs[:, 1], z=eeg_channel_locs[:, 2],
                                mode='markers', marker=dict(color='red', size=5), name='EEG Channels'))

    if megToEEGtransforms is None:
        megToEEGtransforms = [None for i in all_meg_channel_locs]
    
    for meg_channel_locs,megToEEGtransform in zip(all_meg_channel_locs,megToEEGtransforms):
        if megToEEGtransform is not None:
            meg_channel_locs = mne.transforms.apply_trans(megToEEGtransform, meg_channel_locs)
        # Plot MEG channels
        fig.add_trace(go.Scatter3d(x=meg_channel_locs[:, 0], y=meg_channel_locs[:, 1], z=meg_channel_locs[:, 2],
                                mode='markers', marker=dict(color='blue', size=5), name='MEG Channels'))

    # Add labels and legend
    fig.update_layout(title="EEG and MEG Channel Locations on a Simulated Head",
                    scene=dict(xaxis_title='X-axis', yaxis_title='Y-axis', zaxis_title='Z-axis'),
                    legend=dict(x=0, y=1))

    # Adjust view for better visualization
    fig.update_layout(scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)))

    # Export plot to HTML
    fig.write_html(output_name)

In [59]:
plotSensors(all_eeg_channel_locs , all_meg_channel_locs, megToEEGtransforms = all_meg_transforms, sphere = False, output_name= "channel_plot.html")

In [46]:
raw_test = mne.io.read_raw_fif(run, preload=False)
np.array(raw_test['data'][0])[meg_indices]

Opening raw data file /srv/synaptech_openfmri/val/sub-09/run_03.fif...
    Range : 68200 ... 612699 =     62.000 ...   556.999 secs
Ready.


/tmp/ipykernel_400909/2538564231.py:1: RuntimeWarning:

This filename (/srv/synaptech_openfmri/val/sub-09/run_03.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz



array([[ 5.85066307e-12,  5.76385215e-12,  5.73521759e-12, ...,
         2.82236553e-12,  2.34304509e-12,  2.31112916e-12],
       [ 5.09677169e-12,  5.06759194e-12,  5.08160770e-12, ...,
         2.07978593e-12,  1.89984077e-12,  2.05871716e-12],
       [ 4.75392502e-12,  4.69630541e-12,  4.71043261e-12, ...,
         5.30850332e-13,  2.99598766e-13,  4.79035884e-13],
       ...,
       [-2.10656166e-12, -2.23526913e-12, -1.99518296e-12, ...,
        -1.89342936e-12, -1.88969765e-12, -2.33510067e-12],
       [-4.40120473e-12, -4.53279007e-12, -4.23872232e-12, ...,
        -3.49477704e-12, -3.57887665e-12, -4.03370641e-12],
       [-3.79172305e-12, -4.01180308e-12, -3.93098581e-12, ...,
        -3.13948887e-12, -3.19909858e-12, -3.56644402e-12]],
      shape=(102, 544500))

##### Plot for all runs

In [47]:
import numpy as np
import plotly.graph_objects as go


# Create a sphere to simulate the head
phi, theta = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
x = 0.1 * np.sin(theta) * np.cos(phi)  # Radius scaled for visualization
y = 0.1 * np.sin(theta) * np.sin(phi)
z = 0.1 * np.cos(theta)

# Create the plotly figure
fig = go.Figure()

# Plot the head (sphere)
fig.add_trace(go.Surface(x=x, y=y, z=z, colorscale='Blues', opacity=0.1, showscale=False))

for eeg_channel_locs in all_eeg_channel_locs:
    # Plot EEG channels
    fig.add_trace(go.Scatter3d(x=eeg_channel_locs[:, 0], y=eeg_channel_locs[:, 1], z=eeg_channel_locs[:, 2],
                            mode='markers', marker=dict(color='red', size=5), name='EEG Channels'))

for meg_channel_locs in all_meg_channel_locs:
    # Plot MEG channels
    fig.add_trace(go.Scatter3d(x=meg_channel_locs[:, 0], y=meg_channel_locs[:, 1], z=meg_channel_locs[:, 2],
                            mode='markers', marker=dict(color='blue', size=5), name='MEG Channels'))

# Add labels and legend
fig.update_layout(title="EEG and MEG Channel Locations on a Simulated Head",
                  scene=dict(xaxis_title='X-axis', yaxis_title='Y-axis', zaxis_title='Z-axis'),
                  legend=dict(x=0, y=1))

# Adjust view for better visualization
fig.update_layout(scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)))

# Export plot to HTML
fig.write_html("channel_plot.html")

##### Plot for just one run

In [48]:
import numpy as np
import plotly.graph_objects as go


# Create a sphere to simulate the head
phi, theta = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
x = 0.1 * np.sin(theta) * np.cos(phi)  # Radius scaled for visualization
y = 0.1 * np.sin(theta) * np.sin(phi)
z = 0.1 * np.cos(theta)

# Create the plotly figure
fig = go.Figure()

# Plot the head (sphere)
fig.add_trace(go.Surface(x=x, y=y, z=z, colorscale='Blues', opacity=0.1, showscale=False))


# Plot EEG channels
fig.add_trace(go.Scatter3d(x=all_eeg_channel_locs[0][:, 0], y=eeg_channel_locs[:, 1], z=all_eeg_channel_locs[0][:, 2],
                        mode='markers', marker=dict(color='red', size=5), name='EEG Channels'))

# Plot MEG channels
fig.add_trace(go.Scatter3d(x=all_meg_channel_locs[0][:, 0], y=meg_channel_locs[:, 1], z=all_meg_channel_locs[0][:, 2],
                        mode='markers', marker=dict(color='blue', size=5), name='MEG Channels'))

# Add labels and legend
fig.update_layout(title="EEG and MEG Channel Locations on a Simulated Head",
                  scene=dict(xaxis_title='X-axis', yaxis_title='Y-axis', zaxis_title='Z-axis'),
                  legend=dict(x=0, y=1))

# Adjust view for better visualization
fig.update_layout(scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)))

# Export plot to HTML
fig.write_html("channel_plot_just_one.html")

## Finding EEG-MEG Offset Vector (EMOV)
We will take two hard-coded electrodes, one for EEG and one for MEG, and write the logic for consistently calculating their offset vector to feed into the model.

We will take two scalp electrodes, EEG0074 and the MEG electrode at position (0, -0.1086, 0.007) (which turned out to be MEG2111).

In [49]:
### Sanity checks, make sure that order of channel names is always the same
assert all(all_eeg_channel_names[0] == eeg_channel_names for eeg_channel_names in all_eeg_channel_names), "EEG channel names are not equal"
assert all(all_meg_channel_names[0] == meg_channel_names for meg_channel_names in all_meg_channel_names), "MEG channel names are not equal"


### also make sure that MEG locations don't vary
#assert all(np.array_equal(all_meg_channel_locs[0], meg_channel_locs) for meg_channel_locs in all_meg_channel_locs), "MEG locations are not equal"

In [50]:
# # Find the index of the MEG channel location where x, y, and z are close to 0, -0.1086, 0.007
# target_loc = np.array([0, -0.1086, 0.007]) ### these locations were taken from graphical visualization of electrode placement which showed that it is close to EEG0074
# tolerance = 1e-4  # Define a small tolerance for floating point comparison

# # Iterate over all MEG channel locations to find the matching index
# matching_indices = []
# for i, meg_channel_locs in enumerate(all_meg_channel_locs):
#     for j, loc in enumerate(meg_channel_locs):
#         if np.allclose(loc, target_loc, atol=tolerance):
#             matching_indices.append((i, j))

# matching_indices  # This will contain tuples of (run_index, channel_index) where the location matches
# assert all(matching_indices[0][1] == matching_indices_instance[1] for matching_indices_instance in matching_indices), "Channel with that position is not the same"
# matching_index = matching_indices[0][1]
# assert all(all_meg_channel_names[0][matching_index] == meg_channel_names[matching_index] for meg_channel_names in all_meg_channel_names), "MEG channel names at the matching index are not the same across all"
# print(f"MEG channel name with that position is always: {all_meg_channel_names[0][matching_index]}, it is always at MEG channels index {matching_index}, and at all channels index {all_channel_names[0].index(str(all_meg_channel_names[0][matching_index]))}.")

In [51]:
# find the index of EEG channel name EEG074
print(f"EEG0074 channel index is always at EEG channels index {all_eeg_channel_names[0].index("EEG074")}, all channels index {all_channel_names[0].index("EEG074")}")

EEG0074 channel index is always at EEG channels index 73, all channels index 379


In [68]:
_participantRunsDict = train_subject_run_dict
#print(_participantRunsDict)


def find_emov(run_channel_locs, run_meg_eeg_transform):
    """
    Calculate the EEG-MEG Offset Vector (EMOV).

    This function computes the offset vector between a reference EEG electrode
    and a reference MEG electrode after applying a transformation to the MEG location.

    Parameters:
    - run_channel_locs: List or array of channel locations for a specific run.
    - run_meg_eeg_transform: Transformation matrix to be applied to the MEG location.

    Returns:
    - A numpy array representing the offset vector between the EEG and transformed MEG locations.
    """
    eeg_reference_electrode_index = 379
    meg_reference_electrode_index = 236
    
    eeg_loc = run_channel_locs[eeg_reference_electrode_index]
    meg_loc = run_channel_locs[meg_reference_electrode_index]
    meg_loc = mne.transforms.apply_trans(run_meg_eeg_transform, meg_loc)

    return np.array(eeg_loc) - np.array(meg_loc)


print(np.array(all_channel_locs).shape)
print(np.array(all_meg_transforms).shape) 

for run_channel_locs, meg_transform in zip(all_channel_locs, all_meg_transforms):
    print(find_emov(run_channel_locs, meg_transform))

(17, 404, 3)
(17,)
[-0.00674667  0.02095144 -0.04375769]
[-0.00674667  0.02095144 -0.04375769]
[-0.00674667  0.02095144 -0.04375769]
[-0.00674667  0.02095144 -0.04375769]
[-0.00674667  0.02095144 -0.04375769]
[-0.004692    0.03109239 -0.01028339]
[-0.004692    0.03109239 -0.01028339]
[-0.004692    0.03109239 -0.01028339]
[-0.004692    0.03109239 -0.01028339]
[-0.004692    0.03109239 -0.01028339]
[-0.004692    0.03109239 -0.01028339]
[ 0.00152847  0.03342513 -0.01328791]
[ 0.00152847  0.03342513 -0.01328791]
[ 0.00152847  0.03342513 -0.01328791]
[ 0.00152847  0.03342513 -0.01328791]
[ 0.00152847  0.03342513 -0.01328791]
[ 0.00152847  0.03342513 -0.01328791]
